In [64]:
import networkx as nx

In [104]:
class Top:
    ''' A class for representing a finite topological space. 
        ------------
        Attributes |
        ----------------------------------------------------------------
        points: a set {x | x in X } of the points of the top. space
                The elements should all be hashable

        opens: a dictionary of the basic open sets. It is of the form
        { x : { y | x <= y} } where x <= y means x in closure({y})

        closures:   dictionary of the closures of each point
                    { x : cl({x}) }
                    Starts as empty because computing all is O(n^2)
                    Run getClosures() method if all are needed. Else,
                    just call X.cl(x) or X[x] for closure of 1 point
        ----------------------------------------------------------------
    
    '''
    def __init__(self, data, discrete=False):
        # can pass a networkx graph as the data
        if type(data) == nx.DiGraph:
            opens = {}
            for node in data.nodes:
                opens[node] = nx.descendants(data,node) | {node}
            self.opens = opens
            self.points = set(data.nodes)
            self.closures = {}
        elif type(data) == dict:
            self.opens = data
            self.points = set(data)
            self.closures = {}
        
        elif type(data) == int:
            assert data >= 0, 'Topology of integer can\'t be negative'
            self.points = {n for n in range(data)}            
            if not discrete: # order linearly
                self.opens = {n: {k for  k in range(n+1)} 
                              for n in range(data)}
            else: # discrete topology
                self.opens = {n: {n} for n in range(data)}
            self.closures = {}



    def __repr__(self):
        return str(self.opens)
    
    def __len__(self):
        return len(self.points)
    
    def __contains__(self, x):
        return x in self.points
    
    def __iter__(self):
        return iter(self.points)

    def __call__(self, x):
        # Top(x) returns the same as self.opens[x] = {y | y <= x}
        return self.opens[x]
        
    def __getitem__(self, x):
        # Top[x] returns the same as Top.cl(x) or Top.closures[x]
        # This convention mirrors the notation (a,b) for open interval 
        # and [a,b] for closed interval
        try:
            return self.closures[x]
        except:
            self.closures[x] = self.cl(x)
            return self.closures[x]
        
    def __mul__(self, other):
        # product topology
        return self.product(other)
    
    def __add__(self, other):
        # disjoint union topology
        if not self.points & other.points: # already disjoint
            return Top(self.opens | other.opens)
        else: # force them to be disjoint
            point0 = Top({0: {0}})
            point1 = Top({1: {1}})
            return Top((point0 * self).opens | (point1 * other).opens)
        
    def __eq__(self, other):
        return self.opens == other.opens
    
    def __le__(self, other):
        # is a subspace
        if self.points & other.points != self.points:
            return False # isn't a subset
        for point in self.points:
            if other(point) & self.points != self(point):
                return False # not the same open sets
        return True
    
    def __lt__(self, other):
        # proper subspace
        return (not self == other) and self <= other
    
    def __ge__(self, other):
        # contains other as a subspace
        return other <= self
    
    def __gt__(self, other):
        # proper super space
        return (not self == other) and self >= other
    
    def __truediv__(self, subspace):
        # quotient by a subspace
        assert self >= subspace, 'Quotient defined for a subspace'
        quotient = set(self.points - subspace.points)

        # the quotient projection
        def pi(x, pt): 
            if x in self.points - subspace.points:
                return x
            elif x in subspace.points:
                return pt
            else:
                raise ValueError('projection map domain error')
        
        # adjoin a new point not in X \ A, make sure key isn't taken
        pt = '*'; idx = 0
        while pt in quotient:
            idx += 1
            pt = '*' + str(idx)
        # now make the open sets for each x in X \ A
        opens = {x: {pi(y, pt) for y in self(x)} for x in quotient}
        # add in the open set around the new extra point 
        opens[pt] = set()
        for a in subspace.points:
            opens[pt] |= {pi(x, pt) for x in self(a)}
        return Top(opens)
    
    def __invert__(self):
        return self.op()

    def cl(self, x):
        # returns the closure of the singleton {x}
        assert x in self.points,\
        f'{x} is not a point in the topological space'
        close = {y for y in self if x in self(y)}
        # close = { y | x <= y } = { y | x in self.opens[y] }
        return close
        
    def getClosures(self):
        # populates Top.closures for all elements
        for x in self:
            try: # don't bother if it's already computed
                self.closures[x]
            except:
                self.closures[x] = self.cl(x)

    def isleq(self, x, y):
        # partial order defined by x<=y iff x in self.opens[y]
        # or equivalently iff y in self.closures[x]
        return x in self(y)
    
    def isT0(self):
        # True iff self.opens[x] = self.opens[y] implies x = y
        checked = set()
        for x in self:
            for y in self.points - (checked | {x}):
                if self(x) == self(y):
                    return False
            checked |= {x} # don't check U_x=U_y and U_y=U_x separately
        return True
    
    def isT1(self):
        # for finite spaces, this is the same as being discrete
        for x in self:
            if self(x) != {x}:
                return False
        return True
    
    def subspace(self, subset):
        # subspace topology
        assert subset & self.points == subset, 'needs to be a subset'
        return Top({x: self(x) & subset for x in subset})
    
    def product(self, other):
        # cartesian product
        def prod(set1, set2):
            return {(x, y) for x in set1 for y in set2}
        points = prod(self.points, other.points)
        opens = {point : prod(self(point[0]), other(point[1])) \
                 for point in points}
        return Top(opens)
    
    def op(self):
        # reverse the ordering
        self.getClosures()
        op_opens = self.closures
        X_op = Top(op_opens)
        X_op.closures = self.opens
        return (X_op)
    
    def order(self):
        # returns a topological ordering of self.points, meaning that if
        # x <= y, then x will appear first in the list (but converse may 
        # be false if x and y aren't comparable)

        nodes = {x: {'start':0, 'end':0, 'parent':None, 'visited':False}
                 for x in self.points} # nodes for depth first search
        time = 0 
        def dfsVisit(pt): # one iteration of depth first search
            nonlocal time
            time += 1
            node = nodes[pt]
            node['start'] = time
            node['visited'] = True

            for child in self[pt]: # pt <= child
                childnode = nodes[child]
                if childnode['visited'] == False:
                    childnode['parent'] = node
                    dfsVisit(child)
            time += 1
            node['end'] = time
            node['visited'] = 'done'

        # now iterate over all points 
        for pt in self:
            if nodes[pt]['visited'] == False:
                dfsVisit(pt)

        # after depth first search, order the points by finish time
        top_order = sorted(nodes, reverse=True,
                           key=lambda x: nodes[x]['end'] )
        return tuple(top_order)

In [97]:
X = Top({0:{0,1,3}, 1:{1}, 2:{1,2,3}, 3:{3}})
A = X.subspace({0,3})
X.getClosures()
X.closures
X.op().op().order()

(3, 1, 2, 0)

In [105]:
Y = Top(3)
(Y.op() *  Y).order()

((2, 0), (1, 0), (0, 0), (2, 1), (2, 2), (1, 1), (1, 2), (0, 1), (0, 2))

In [82]:
a = {1:'a', 2:'d', 0:'c'}
sorted(a, key=lambda x: a[x])

[1, 0, 2]

In [78]:
d = {'a': {'a':1, 'b':2, 'c':3}}
d['a']['b']

2